## <center>Hướng dẫn convert môi trường Miner sang môi trường Tensorflow Agents


Tensorflow hiện tại là một trong những framework được sử dụng rộng rãi nhất trong lĩnh vực Deep Learning. Khi google cho ra đời phiên bản Tensorflow 2.0, Tensorflow đã không còn chỉ là một công cụ để tạo ra các cấu trúc học sâu, mà nó giờ đã trở thành một hệ sinh thái với rất nhiều tính năng khác nhau. Hôm nay chúng ta sẽ nói đến một modulde của Tensorflow dành riêng cho Reinforcement Learning, Tensorflow Agents.


Khi sử dụng bất kì thư viện gì thì việc đầu tiên chúng ta quan tâm chính là làm sao để áp dụng những thuật toán đã có sẵn của thư viện đó vào bài toán của chúng ta, hoặc nói cách khác là định dạng lại bài toán mà chúng ta cần giải quyết sao cho thư viện có thể hiểu và giải quyết bài toán đó.

Trong Reinforcement Learning, các vấn đề cần giải quyết sẽ được mô hình hóa dưới dạng một môi trường có thể tương tác được, ví dụ như bài toán tìm đường trong một mê cung thì phải có môi trường là một mê cung. Vậy hôm nay mình sẽ chia sẻ với các bạn bước đầu tiên, và cũng không kém phần quan trọng khi giải quyết bài toán Reinforcement Learning - tạo môi trường với công cụ TF Agents

Đầu tiên chúng ta sẽ import các thư viện cần thiết để convert môi trường từ định dạng Object của Python sang định dạng chuẩn của TF Agents

In [ ]:
import numpy as np

from tf_agents.environments import py_environment as pyenv, \
                                 tf_py_environment, utils
from tf_agents.specs import array_spec 
from tf_agents.trajectories import time_step

Ở đây giải thích một chút về các module được mình import vào. Lần đầu tiếp xúc với tensorflow Agent mình cũng khá là choáng khi một file đơn giản lại phải import nhiều module như vậy. Chẳng bù với khi dùng module Tensorflow 2.0 thì chỉ cần import tensorflow và tensorflow.keras. 

Để tránh cho các bạn bị choáng như mình thì mình xin giới thiệu qua các module:

- **tf_agents.environments** là module chứa các object và function để tạo môi trường theo chuẩn TF Agents. 
- **tf_agents.specs** là module định nghĩa các cấu trúc dữ liệu dùng trong TF Agents, nếu bạn nào dùng Gym API quen thì nó sẽ giống như action_space và observation_space (Box, Discrete ,...) trong Gym.
- **tf_agents.trajectories** là module chứa object TimeStep, là một tuple bao gồm (StepType, Observation, Reward, Discount Factor)

In [ ]:
from MinerEnv import MinerEnv

Thật may mắn là chúng ta không cần phải tạo lại môi trường từ đầu, chúng ta sẽ dùng lại môi trường được cung cấp bởi BTC, format lại cho môi trường có thể làm việc được với TF Agents

In [ ]:
MAP_MAX_X = 21 #Width of the Map
MAP_MAX_Y = 9  #Height of the Map

Tiếp theo chúng ta định nghĩa chiều dài và chiều rộng map, theo như chia sẻ của BTC thì kích thước bản đồ sẽ không đổi, nên chúng ta không cần lo lắng khi set cứng thông số này.

In [ ]:
class TFAgentsMiner(pyenv.PyEnvironment):
    def __init__(self, host, port, debug = False):
        super(TFAgentsMiner, self).__init__()

        self.miner_env= MinerEnv(host, port)
        self.miner_env.start()
        self.debug = debug
        
        self._action_spec = array_spec.BoundedArraySpec(shape = (), dtype = np.int32, minimum = 0, maximum = 5, name = 'action')
        self._observation_spec = array_spec.BoundedArraySpec(shape = (MAP_MAX_X*5,MAP_MAX_Y*5,6), 
                dtype = np.float32, name = 'observation')

Bước tiếp theo chúng ta định nghĩa môi trường mới sẽ là class con của pyEnvironment. Trong TF Agents, có hai loại môi trường. Một là môi trường định nghĩa theo cú pháp của Python - pyEnvironment, và môi trường định nghĩa theo cú pháp của Tensorflow - TFEnvironment. Môi trường Python thì dễ định nghĩa, trong khi môi trường TF định nghĩa rất phức tạp. Tuy nhiên môi trường TF giống như một static graph, điều này giúp cho môi trường TF hoạt động nhanh hơn môi trường Python nhiều, đồng thời với môi trường TF, chúng ta có thể tiến hành parallel training một agent trên nhiều môi trường cùng một lúc


Vì TF Agents hỗ trợ API để chuyển qua lại giữ pyEnvironment và TFEnvironment nên workflow hiệu quả nhất hiện tại sẽ là định nghĩa môi trường pyEnvironmen, sau đó dùng hàm **tf_py_environment(...)** để chuyển thành TFEnvironment 

Để ý trong hàm \_\_init\_\_ chúng ta khai báo thông tin của observation, cũng như là thông tin của action space dưới dạng một BoundedArraySpec- có nghĩa là một mảng bị chặn trên và dưới. Đồng thời chúng ta cũng khởi tạo môi trường MinerEnv được cung cấp bởi ban tổ chức để sử dụng lại.

In [ ]:
    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

Tiếp theo chúng ta phải định nghĩa hai hàm action_spec và observation_spec trả về hai protected variables \_action\_spec và \_observation\_spec mà ta đã định nghĩa trong hàm \_\_init\_\_

Việc định nghĩa hai hàm này là bắt buộc vì trong class cha pyEnvironment, hai hàm này được định nghĩa là hai @abstractmethod

In [ ]:
    def _reset(self):
        mapID = np.random.randint(1, 6)
        posID_x = np.random.randint(MAP_MAX_X)
        posID_y = np.random.randint(MAP_MAX_Y)
        request = ("map" + str(mapID) + "," + str(posID_x) + "," + str(posID_y) + ",50,100")
        self.miner_env.send_map_info(request)
        self.miner_env.reset()
        observation = self.miner_env.get_state()

        return time_step.restart(observation)

Hàm \_reset() ở trên là một hàm protected, nên sẽ không được gọi trực tiếp mà sẽ được gọi thông qua env.reset()

Trong hàm trên chúng ta sẽ đơn giản gửi một request để lấy thông tin map mới, đồng thời random lại vị  trí của người chơi và lấy ra observation đầu tiên.

Một điểm cần chú ý là chúng ta không trả về thẳng observation mà sẽ wrap nó bằng object TimeStep của TF Agents.

TimeStep có thể hiểu đơn giản là một tuple (StepType, Observation, Reward, Discount factor). Trong đó StepType bao gồm:
- FIRST: chỉ thị đây là observation đầu tiên (khởi điểm) của một trajectory. TimeStep có type FIRST thì không có Reward và Discount Factor 
- MID: chỉ thị đây là observation chuyển tiếp (nằm ở giữa) của một trajectory. 
- LAST: chỉ thị đây là observation cuối cùng của một trajectory. Có thể dùng LAST type như chỉ thị done = True, kết thúc một trajectory/episode. TimeStep có type LAST thì không có Discount Factor.

Việc TimeStep mang Type nào thì sẽ do function dùng để wrap quyết định. Cụ thể:
- time_step.restart(...) -> StepType.FIRST
- time_step.transition(...) -> StepType.MID
- time_step.termination(...) -> StepType.LAST

Các bạn có thể thấy trong hàm \_reset() mình dùng wrapper time_step.restart(...) để chỉ thị đây là điểm bắt đầu một trajectory mới.


In [ ]:
    def _log_info(self):
        info = self.miner_env.socket

        # print(f'Map size:{self.info.user.max_x, self.miner_env.state.mapInfo.max_y}')
        print(f"Self  - Pos ({info.user.posx}, {info.user.posy}) - Energy {info.user.energy} - Status {info.user.status}")
        for bot in info.bots:
            print(f"Enemy  - Pos ({bot.info.posx}, {bot.info.posy}) - Energy {bot.info.energy} - Status {bot.info.status}")

Hàm \_log\_info(...) chủ yếu dùng để in vị trí người chơi phục vụ cho việc debug.

In [ ]:
    def _step(self, action):
        if self.debug:
            self._log_info()

        self.miner_env.step(str(action))
        observation = self.miner_env.get_state()
        reward = self.miner_env.get_reward()

        if not self.miner_env.check_terminate():
            return time_step.transition(observation, reward)
        else:
            self.reset()
            return time_step.termination(observation, reward)

Tiếp theo chúng ta định nghĩa hàm \_step(...) thể hiện sự thay đổi của môi trường dưới tác động của agent. Hàm này nhận vào một hành động action, và trả về observation và reward tương ứng. Tương tự như hàm \_reset(...), mình cũng dùng wrapper để trả về định dạng TimeStep. 

Các bạn có thể thấy mình kiểm tra xem MinerEnv đã cho tính hiệu kết thúc hay chưa, nếu đã kết thúc thì mình dùng wrapper time_step.termination(...) và reset lại môi trường, còn chưa kết thúc thì mình dùng wrapper time_step.transition(...)

In [ ]:
    def render(self):
        pass

Thông thường đối với các môi trường khác sẽ cần hàm render(...) để user có thể xem trực tiếp agent hoạt động như thế nào. Trong trường hợp MinerEnv của chúng ta chưa hỗ trợ đồ họa nên mình sẽ bỏ qua.

In [ ]:
if __name__ == '__main__':
    env = TFAgentsMiner("localhost", 1111)
    utils.validate_py_environment(env, episodes=5)

Một điều mình rất thích ở TF Agent đó là họ cung cấp cho người dùng công cụ để kiểm tra môi trường được định nghĩa đã hoạt động tốt hay chưa. Sau khi định nghĩa xong môi trường của mình hãy dùng hàm utils.validate_py_environment(...) để kiểm tra. Hàm này sẽ dùng một Ramdom Policy để kiểm tra môi trường của bạn, đồng thời so sánh các Specs (Action Spec, Observation Spec, Time Spec,...) xem đã phù hợp chưa.